# ベイズ統計の理論と方法

## はじめに
参考書籍１のp1８の事後分布の例のプロットの出力を目指す。

初めは、nbでわちゃわちゃ書いて、ある程度固まったら、スクリプトにまとめる方式でやる。

他にも詳しくやってる人がいるので詳しくはそちらを参考に。(juliaでやってる人とかいる）

あくまでも自分の勉強としてやっていく。

nbはgoogle colabを使用。


In [0]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import scipy.stats as ss

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

plt.style.use("ggplot")

%matplotlib inline

関数$\mathcal{N(x)}$ を平均０で分散が１の正規分布とする

$$\mathcal{N(x)} = \frac{1}{\sqrt{2\pi}} exp ( - \frac{x^2}{2}) \tag{1.26}$$

として、パラメータ$w$の集合を$w = \{ (a, b)| 0 \leq a \leq 1 , b \in \mathbb{R}^1 \}$をもつ確率モデルとして、混合ガウス分布モデルを考える

$$p(x|w) = a \mathcal{N(x)} + (1+a)\mathcal{N(x-b)} \tag{1.27}$$

を考えていく。

この確率分布モデルのパラメータの事後分布の形を計算して三次元プロットを試みる。

以下では、パラメータの集合は、

$$W = \{ (a, b)| 0 \leq a \leq 1 , |b| \leq 5  \} \tag{1.28}$$とする

また事前分布は、一様分布を考え

$$\phi(w) = \frac{1}{10}\tag{1.29}$$

このとき事後分布は尤度関数と同じ形状をしていてる。

書籍に即して、$(a_0, b_0) = (0.5, 3.0) , (0.5 , 1.0) , (0.5, 0.5)$

のパターンを試す。サンプルの個数はn=100

In [0]:
def normal_pdf(x):
    pdf =  ( 1 / np.sqrt(2 * np.pi) ) * np.exp( - (1 / 2) * x ** 2)
    return pdf

def mixture_normal(x, a, b):
    return (1 - a) * normal_pdf(x) + a * normal_pdf(x-b)

## 混合ガウス分布の確率密度関数

In [0]:
x = np.linspace(-10, 10, 100)

y = mixture_normal(x, a=0.5, b=3.0)
# y = normal_pdf(x)

In [0]:
plt.figure(figsize=(10, 8))
plt.plot(x, y, marker="o")
plt.xlabel("x")
plt.ylabel("pdf")
plt.show()

## 事後分布のプロット

In [0]:
def log_likelihood(x, a, b ):
    def normal_pdf(x):
        pdf =  ( 1 / np.sqrt(2 * np.pi) ) * np.exp( - (1 / 2) * x ** 2)
        return pdf
    def mixture_normal(x, a, b):
        return (1 - a) * normal_pdf(x) + a * normal_pdf(x-b)
    return np.sum(np.log(mixture_normal(x, a, b)), axis=0)

真の分布を$q(x) = p(x|a_0, b_0)$として

1. まず、$(a_0, b_0) = (0.5, 3.0)$の時を考える



In [0]:
# 真の分布からのデータ
def q_x(a, b, sample_size):
    return (1 - a ) * np.random.normal(size=sample_size) + a * np.random.normal(loc=b, size=sample_size)

# W = (a_0, b_0)
W = [(0.5, 3.0), (0,5, 1.0), (0.5, 0.5)]

In [0]:
data1 = q_x(a=W[0][0], b=W[0][1], sample_size=200)

In [0]:
data1.shape

In [0]:
data = data1
def calc_post(data):
    a_i = np.linspace(0.0, 1.0, num=data.shape[0])
    b_i = np.linspace(-5.0, 5.0, num=data.shape[0])
    # data1の時の混合ガウス分布の確率密度関数
    a, b = np.meshgrid(a_i, b_i)
    posteriors = [[0]*data.shape[0] for _ in range(data.shape[0])]
    for c , i in enumerate(a_i):
        for r, j in enumerate(b_i):
            posterior_ = np.exp(log_likelihood(data, a=i, b=j)) * 1e+100
            posteriors[c][r] = posterior_

    post = np.array(posteriors)
    return post, a, b

post, a, b = calc_post(data)
print(post.shape, a.shape, b.shape)
fig = plt.figure(figsize=(16,10))
ax = fig.gca(projection='3d')

# Plot the surface.
surf = ax.plot_surface(a, b, post, cmap=cm.coolwarm,
                    linewidth=0, antialiased=False)

# Customize the z axis.
ax.set_zlim(0, 1.01)
ax.set_ylim(5.01, -5.01)
ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()    


In [0]:
import math

In [0]:
def plot_posterior(data, scaling):
    a_i = np.linspace(0.0, 1.0, num=data.shape[0])
    b_i = np.linspace(-5.0, 5.0, num=data.shape[0])
    # data1の時の混合ガウス分布の確率密度関数
    a, b = np.meshgrid(a_i, b_i)
    posteriors = [[0]*data.shape[0] for _ in range(data.shape[0])]
    for c , i in enumerate(a_i):
        for r, j in enumerate(b_i):
            posterior_ = np.exp(log_likelihood(data, a=i, b=j))
            data_size = np.log10(posterior_)
            posteriors[c][r] = posterior_ * data_size
    
    post = np.array(posteriors)
    fig = plt.figure(figsize=(16,10))
    ax = fig.gca(projection='3d')

    # Plot the surface.
    surf = ax.plot_surface(a, b, post, cmap=cm.coolwarm, linewidth=0, antialiased=False)

    # Customize the z axis.
    ax.set_zlim(0, 1.01)
    ax.set_ylim(5.01, -5.01)
    ax.zaxis.set_major_locator(LinearLocator(10))
    ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

    # Add a color bar which maps values to colors.
    fig.colorbar(surf, shrink=0.5, aspect=5)

    plt.show()    


In [0]:
scaling = 1
plot_posterior(data1, scaling)

In [0]:
data2 = q_x(a=W[1][0], b=W[1][1], sample_size=100)

In [0]:
scaling2 = 1e+62
plot_posterior(data2, scaling=scaling2)

In [0]:
data3 = q_x(a=W[2][0], b=W[2][1], sample_size=100)

In [0]:
scaling3 = 1e+50
plot_posterior(data3, scaling=scaling3)

In [0]:
import chart_studio.plotly as py
import plotly.graph_objs as go

def plotly_post(data, a, b):
    data = [
        go.Surface(
            x=a,
            y=b,
            z=post
        )
    ]
    layout = go.Layout(
        title="posterior",
        autosize=False,
        width=500,
        height=500,
        margin=dict(
            l=65,
            r=50,
            b=65,
            t=90
        )
    )
    fig = go.Figure(data=data, layout=layout)
    fig.show()

In [0]:
post, a, b = calc_post(data2)
plotly_post(post, a, b)

In [0]:
post, a, b = calc_post(data3)
plotly_post(post, a, b)

# 参考

- 三次元のプロット

[3次元グラフの書き方（matplotlib, pyplot, mplot3d, MPL） ; Qiita](https://qiita.com/orange_u/items/8a1e285a45093857aef7)